In [998]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO


import pandas as pd
import numpy as np
import re
# import pycountry
from textblob import TextBlob
from calendar import month_name
from collections import Counter
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')


In [170]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb:pdf')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [ ]:
import glob

text_lst = []
for i,filename in enumerate(glob.glob('PAB/*.pdf')):
    try: 
        text_lst.append(convert_pdf_to_txt(filename))
        print("Successful read file: ", filename)
    except:
        print("Can't read file: ", filename)

In [269]:
len(text_lst)

16

In [336]:
text_lst[1]

'National Transportation Safety Board\n\nWashington, D.C. 20594\n\nPipeline Accident Brief\n\nDCA99-MP005\nColonial Pipeline Company\nHazardous liquid petroleum products pipeline\nPipe rupture\nKnoxville, Tennessee\nDiesel fuel, high sulfur, 86 Grade\n11:58 p.m. EST \nFebruary 9, 1999\nNone\n$7 million \nAbout 53,550 gallons (1,275 barrels)\nApproximately 91 psig\n10-inch-diameter, Grade 5L X-42, 0.250-inch wall \nthickness, electric resistance weld steel pipe\n\nR A SP\n\nN\n \n\nO\n\nL U RIBUS UNUM \n\nE P\n\n \n\n \n\nA L   T\nN\nO\nI\nT\nA\nN\n\n \n\nS\n\nR\n\nT\n\nA\n\nT\n\nI\n\nO\nN\n\n \n\nR D\n\nA\n\nF\n\nETY\n\n B O A\n\nAccident Number:\nOwner/Operator:\nType of System:\nAccident Type:\nLocation:\nProduct Released:\nTime:\nDate:\nFatalities/Injuries:\nProperty Damage:\nQuantity Released:\nPipeline Pressure:\nComponent Affected:\n\nThe Accident\n\nOn  the  afternoon  of  February  9,  1999,  Colonial  Pipeline  Company  (Colonial)\nsuccessfully  completed  a  delivery  of  die

In [670]:
str(text_lst[1])

'p'

In [695]:
# text0 = "".join(text_lst[4]).split("\n")
# text0 = "\n".join(text0)
text0 = re.sub(r'\n', "-", text_lst[6])
text0

'R A SP--N- --O--E P L U RIBUS UNUM --R--T--A--T--I--O-N--NAL  T-- --O-I-T-A-N-- --S--AFETY-- -- -- B O A R D--National Transportation Safety Board --Washington, D.C. 20594 --Pipeline Accident Brief -- -- -- - - - - - -- - - - - - --Accident No.:   -Type of System: -Accident Type: - -Location: - -Date: -Time:  - -Owner/Operator: - -Fatalities: -Injuries: - -Property Damage and Losses:  - -Material Released: - - -Pipeline Pressure: - -Component Affected:   - -- - -- - --DCA-06-MP-001 -Natural gas distribution -Pipeline break, explosion, and fire -Bergenfield, New Jersey -December 13, 2005 -9:26 a.m. eastern standard time1 -Public Service Electric and Gas Company -3 -5 -$863,300 -Natural gas -11 1/2 pounds per square inch, gauge  -1 1/4-inch steel service line --The Accident --On  December  13,  2005,  at  9:26  a.m.,  an  apartment  building  exploded  in  Bergenfield, -New Jersey, after natural gas migrated into the building from a damaged pipeline. Investigators -found a break in an u

In [763]:
accident_detail = re.sub(r'\n', "@", text_lst[8])
accident_detail = re.findall(r'(?<=The Accident).*?\w+\. @@', accident_detail)
accident_detail[0]

' @ @@On March 5, 2008, about 1:39 p.m., a natural gas explosion destroyed a residence @at  171  Mardi  Gras  Drive  in  Plum  Borough,  Pennsylvania,  killing  a  man  and  seriously @injuring  a  4-year-old  girl.  Two  other  houses  were  destroyed,  and  11  houses  were @damaged. Property damage and losses were $1,000,000. @Postaccident Investigation @@No report of a gas odor was filed with Dominion Peoples Natural Gas Company @(Dominion) before the accident. Several neighbors and a postal worker who were on the @property 30 minutes before the explosion stated during postaccident interviews that they @had  not  smelled  gas  before  the  accident. The  Pennsylvania  Public  Utility  Commission @(PUC) tested several homeowners to determine whether they could smell the odorant that @indicates  the  presence  of  natural  gas;  each  was  able  to  smell  the  odorant  at  minimum @levels.  @@                                                 @@1 All times in this brief are eastern st

## Show feature names from text file

In [867]:
def metric_filter(text):
    '''Collect only wanted metrics that are found in the text document'''
    lst = []
    flag = False
    for metric in text:
        metric = metric.replace(':',"").replace('.',"").replace(" ","_").replace("/","_").lower()
        metric = metric.replace('accident_number','accident_no')
        metric = metric.replace('property_damage_and_losses','property_damage')
        metric = metric.replace('accident_type','type_of_accident')
        if flag == False:
            lst.append(metric)
        if metric == 'component_affected':
            flag = True
    if len(lst) > 10 and len(lst) < 15:
        lst.append('Text')
        return lst

In [868]:
def process_texts(text_lst):
    '''Return 2D array, each represents for a list of tuple (features, index)'''
    metrics = []
    features = []
    for index, textfile in enumerate(text_lst):
        t = re.findall(r'[A-Z].*\:', textfile)
        
        features = metric_filter(t)
#         features.append('Text')
        if features is not None:
            metrics.append((features, index))
        
    return metrics

In [869]:
metrics = process_texts(text_lst)
for i in metrics:
    print(len(i[0]))

14
14
13
13
14
15
14
14
13
12


In [870]:
metrics

[(['accident_no',
   'type_of_system',
   'type_of_accident',
   'location',
   'date_and_time',
   'owner_operator',
   'property_damage',
   'fatalities',
   'injuries',
   'material_released',
   'pressure',
   'type_of_failure',
   'component_affected',
   'Text'],
  0),
 (['accident_no',
   'owner_operator',
   'type_of_system',
   'type_of_accident',
   'location',
   'product_released',
   'time',
   'date',
   'fatalities_injuries',
   'property_damage',
   'quantity_released',
   'pipeline_pressure',
   'component_affected',
   'Text'],
  1),
 (['pipeline_accident_no',
   'type_of_system',
   'type_of_accident',
   'location',
   'date_and_time',
   'owner_operator',
   'fatalities_injuries',
   'damage_clean_up_cost',
   'material_released',
   'quantity_released',
   'pipeline_pressure',
   'component_affected',
   'Text'],
  2),
 (['pipeline_accident_no',
   'type_of_system',
   'type_of_accident',
   'location',
   'date_and_time',
   'owner_operator',
   'fatalities_injur

In [984]:
def get_metric_value(text):
    '''Return a list of metrics in the document'''
    lst = []
    t = re.findall(r'DCA.*@@', text)
    
    try:
        t = t[0].split('@@')[0]
        t = t.split('@')
        accident_detail = re.sub(r'@', "-", text)
        accident_detail = re.findall(r'(?<=The Accident).*?\w+\. --', accident_detail)
#         accident_detail = accident_detail[0]
        t.append(str(accident_detail).encode('utf-8'))
        if t[-1] == []:
            t[-1] = 'None'
        
        if len(t) > 0 :
            lst.append((t,i))
    except:
        print("Can't extract data at file: ", i)
    return lst

In [985]:
infos = []
for i,textfile in enumerate(text_lst):
    text = textfile.replace("\n","@")
    infos.append(get_metric_value(text))

("Can't extract data at file: ", 13)
("Can't extract data at file: ", 14)
("Can't extract data at file: ", 15)


In [1001]:
infos[1][0][0][-1]

"['--On  the  afternoon  of  February  9,  1999,  Colonial  Pipeline  Company  (Colonial)-successfully  completed  a  delivery  of  diesel  fuel  to  the  East  Knoxville  delivery  facility-(East Knoxville) in Knoxville, Tennessee, using its Atlanta Junction/Knoxville stubline.1-To  initiate  the  delivery,  an  operator  at  Colonial\\xe2\\x80\\x99s  Knoxville  Terminal  began  operating-valves  that  directed  product  into  a  10-inch-diameter  steel  pipeline  leading  to  the  East-Knoxville facility, about 8 miles away. At about the same time, a valve was opened at the-other end of that line to allow the product to flow into the East Knoxville facility tanks. At-the completion of the delivery, the valves were closed, which left the pipeline filled with-diesel fuel at a pressure of about 91 psig. --']"

In [987]:
def metric_value_bonder(metrics, infos):
    dfs = []
    for i in range(len(metrics)):
        info_lst = []
        metric_lst = []
        if metrics[i][1] == infos[i][0][1]:
            for j,value in enumerate(metrics[i][0]):
#                 print(value)
                if value == 'Text':
                    info_value = infos[i][0][0][-1]
                else:
                    info_value = infos[i][0][0][j]
                info_lst.append(info_value)
                metric_lst.append(value)
            df = pd.DataFrame(data=[info_lst], columns=metric_lst)
            dfs.append(df)
    return pd.concat(dfs, axis=0)

In [988]:
df = metric_value_bonder(metrics, infos)
df

,Text,accident_no,component_affected,damage_clean_up_cost,date,date_and_time,failure_type,fatalities,fatalities_injuries,injuries,...,pipeline_pressure,pressure,product_released,property_damage,quantity_released,system_type,time,type_of_accident,type_of_failure,type_of_system
0,[],DCA-99-MP-004,¾-inch steel gas service line,NaN,NaN,"January 22, 1999, 10:02 a.m. CST",NaN,3,NaN,6,...,NaN,35 psig,NaN,$1.4 million,NaN,NaN,NaN,"Rupture, explosion, and fire",Excavation damage,Natural gas service line
0,"['--On the afternoon of February 9, 1999...",DCA99-MP005,"10-inch-diameter, Grade 5L X-42, 0.250-inch wall",NaN,"February 9, 1999",NaN,NaN,NaN,None,NaN,...,Approximately 91 psig,NaN,"Diesel fuel, high sulfur, 86 Grade",$7 million,"About 53,550 gallons (1,275 barrels)",NaN,11:58 p.m. EST,Pipe rupture,NaN,Hazardous liquid petroleum products pipeline
0,"[' --About 12:12 p.m. CST1 on January 27, 2000...",NaN,"606 pounds per square inch, gauge, (psig)",None,NaN,"January 27, 2000, 12:12 p.m. central standard",NaN,NaN,Marathon Ashland Pipe Line LLC,NaN,...,"About 489,000 gallons",NaN,NaN,NaN,Crude oil,NaN,NaN,Pipe failure and leak,NaN,Hazardous liquid
0,"[' --On March 9, 2000, about 10:20 p.m., centr...",NaN,at site of failure,$18 million,NaN,"March 9, 2000, at 10:20 p.m., central standard...",NaN,NaN,None,NaN,...,"705 pounds per square inch, gauge, (psig)",NaN,NaN,NaN,"Approximately 564,000 gallons",NaN,NaN,Pipe failure and leak,NaN,Hazardous liquid
0,"[' --On July 2, 2003, a contractor hired...",DCA-03-MP-004,Excavation damage,NaN,NaN,"July 2, 2003, at 1:44 p.m., eastern daylight t...",NaN,business as Conectiv Power Delivery,NaN,None,...,NaN,Natural gas,NaN,14,NaN,NaN,NaN,"Rupture, explosion, and fire",6 to 8 inches water column,Natural gas distribution
0,"[' --On August 21, 2004, about 8:54 a.m....",DCA-04-MP-006,2-inch plastic main line,NaN,"August 21, 2004",NaN,Butt-fusion joint leak,2,NaN,None,...,NaN,"50 pounds per square inch, gauge",NaN,"$800,000",NaN,Natural gas distribution,About 8:54 a.m. eastern daylight time1,"Leak, explosion, and fire",NaN,NaN
0,"[' --On December 13, 2005, at 9:26 a.m.,...",DCA-06-MP-001,1 1/4-inch steel service line,NaN,"December 13, 2005",NaN,NaN,3,NaN,5,...,"11 1/2 pounds per square inch, gauge",NaN,NaN,"$863,300",NaN,NaN,9:26 a.m. eastern standard time1,"Pipeline break, explosion, and fire",NaN,Natural gas distribution


In [989]:
df.rename(columns = {'data_and_time'  :'Incident Date',
                     'NAME'   : 'Name of Owner',
                     'ACCST': 'State',
                     'accident_type'   : 'Cause of Loss', 
                      'MANU'  : 'Name of Installation',
                      'fatalities_injuries' : 'Injury',
                      'fatalities' : 'Death',
                     'property_damage'  : 'Loss Estimated ($)'},
                      inplace = True)

In [990]:
df = df.reset_index()
df

,index,Text,accident_no,component_affected,damage_clean_up_cost,date,date_and_time,failure_type,Death,Injury,...,pipeline_pressure,pressure,product_released,Loss Estimated ($),quantity_released,system_type,time,type_of_accident,type_of_failure,type_of_system
0,0,[],DCA-99-MP-004,¾-inch steel gas service line,NaN,NaN,"January 22, 1999, 10:02 a.m. CST",NaN,3,NaN,...,NaN,35 psig,NaN,$1.4 million,NaN,NaN,NaN,"Rupture, explosion, and fire",Excavation damage,Natural gas service line
1,0,"['--On the afternoon of February 9, 1999...",DCA99-MP005,"10-inch-diameter, Grade 5L X-42, 0.250-inch wall",NaN,"February 9, 1999",NaN,NaN,NaN,None,...,Approximately 91 psig,NaN,"Diesel fuel, high sulfur, 86 Grade",$7 million,"About 53,550 gallons (1,275 barrels)",NaN,11:58 p.m. EST,Pipe rupture,NaN,Hazardous liquid petroleum products pipeline
2,0,"[' --About 12:12 p.m. CST1 on January 27, 2000...",NaN,"606 pounds per square inch, gauge, (psig)",None,NaN,"January 27, 2000, 12:12 p.m. central standard",NaN,NaN,Marathon Ashland Pipe Line LLC,...,"About 489,000 gallons",NaN,NaN,NaN,Crude oil,NaN,NaN,Pipe failure and leak,NaN,Hazardous liquid
3,0,"[' --On March 9, 2000, about 10:20 p.m., centr...",NaN,at site of failure,$18 million,NaN,"March 9, 2000, at 10:20 p.m., central standard...",NaN,NaN,None,...,"705 pounds per square inch, gauge, (psig)",NaN,NaN,NaN,"Approximately 564,000 gallons",NaN,NaN,Pipe failure and leak,NaN,Hazardous liquid
4,0,"[' --On July 2, 2003, a contractor hired...",DCA-03-MP-004,Excavation damage,NaN,NaN,"July 2, 2003, at 1:44 p.m., eastern daylight t...",NaN,business as Conectiv Power Delivery,NaN,...,NaN,Natural gas,NaN,14,NaN,NaN,NaN,"Rupture, explosion, and fire",6 to 8 inches water column,Natural gas distribution
5,0,"[' --On August 21, 2004, about 8:54 a.m....",DCA-04-MP-006,2-inch plastic main line,NaN,"August 21, 2004",NaN,Butt-fusion joint leak,2,NaN,...,NaN,"50 pounds per square inch, gauge",NaN,"$800,000",NaN,Natural gas distribution,About 8:54 a.m. eastern daylight time1,"Leak, explosion, and fire",NaN,NaN
6,0,"[' --On December 13, 2005, at 9:26 a.m.,...",DCA-06-MP-001,1 1/4-inch steel service line,NaN,"December 13, 2005",NaN,NaN,3,NaN,...,"11 1/2 pounds per square inch, gauge",NaN,NaN,"$863,300",NaN,NaN,9:26 a.m. eastern standard time1,"Pipeline break, explosion, and fire",NaN,Natural gas distribution


In [991]:
df.drop('index', axis=1, inplace=True)

In [992]:
df['location'] = df['location'].apply(lambda x: re.findall(r'(?<=, )[A-Z].*', x))

In [993]:
df['location'] = df['location'].apply(lambda x: str(x).replace("[","").replace("]","").replace("'","").replace(" ",""))
df['Text'] = df['Text'].apply(lambda x: str(x).replace("[","").replace("]","").replace("'","").replace("@","").replace("-",""))

In [994]:
df['Text'][5]

' On  August  21,  2004,  about  8:54  a.m.,  a  natural  gas  explosion  destroyed  a  residence located  at  48  Woodland  Lane  in  DuBois,  Pennsylvania.  The  two  residents  were  killed  in  this accident. (See figure 1.) Postaccident Investigation National Fuel Gas Distribution Corporation (National Fuel) probed the ground near the residence for gas leaks. Combustible gastoair readings were found in two areas: one area was adjacent to the affected residence, and the other area was directly over the failed pipe on the front lawn.  After  the  explosion,  National  Fuel  excavated  the  front  lawn.  The  excavation  revealed  a leaking buttfusion joint in a 2inchdiameter plastic main line pipe. (See figure 2.) A buttfusion joint is made by the following process. A heater plate is inserted between two pipe ends, melting the material on each end. The heater is then removed, and the pipe ends are pushed together under applied force. The pipe ends remain under applied force as the p

In [995]:
df

,Text,accident_no,component_affected,damage_clean_up_cost,date,date_and_time,failure_type,Death,Injury,injuries,...,pipeline_pressure,pressure,product_released,Loss Estimated ($),quantity_released,system_type,time,type_of_accident,type_of_failure,type_of_system
0,,DCA-99-MP-004,¾-inch steel gas service line,NaN,NaN,"January 22, 1999, 10:02 a.m. CST",NaN,3,NaN,6,...,NaN,35 psig,NaN,$1.4 million,NaN,NaN,NaN,"Rupture, explosion, and fire",Excavation damage,Natural gas service line
1,"On the afternoon of February 9, 1999, C...",DCA99-MP005,"10-inch-diameter, Grade 5L X-42, 0.250-inch wall",NaN,"February 9, 1999",NaN,NaN,NaN,None,NaN,...,Approximately 91 psig,NaN,"Diesel fuel, high sulfur, 86 Grade",$7 million,"About 53,550 gallons (1,275 barrels)",NaN,11:58 p.m. EST,Pipe rupture,NaN,Hazardous liquid petroleum products pipeline
2,"About 12:12 p.m. CST1 on January 27, 2000, a ...",NaN,"606 pounds per square inch, gauge, (psig)",None,NaN,"January 27, 2000, 12:12 p.m. central standard",NaN,NaN,Marathon Ashland Pipe Line LLC,NaN,...,"About 489,000 gallons",NaN,NaN,NaN,Crude oil,NaN,NaN,Pipe failure and leak,NaN,Hazardous liquid
3,"On March 9, 2000, about 10:20 p.m., central s...",NaN,at site of failure,$18 million,NaN,"March 9, 2000, at 10:20 p.m., central standard...",NaN,NaN,None,NaN,...,"705 pounds per square inch, gauge, (psig)",NaN,NaN,NaN,"Approximately 564,000 gallons",NaN,NaN,Pipe failure and leak,NaN,Hazardous liquid
4,"On July 2, 2003, a contractor hired by...",DCA-03-MP-004,Excavation damage,NaN,NaN,"July 2, 2003, at 1:44 p.m., eastern daylight t...",NaN,business as Conectiv Power Delivery,NaN,None,...,NaN,Natural gas,NaN,14,NaN,NaN,NaN,"Rupture, explosion, and fire",6 to 8 inches water column,Natural gas distribution
5,"On August 21, 2004, about 8:54 a.m., a...",DCA-04-MP-006,2-inch plastic main line,NaN,"August 21, 2004",NaN,Butt-fusion joint leak,2,NaN,None,...,NaN,"50 pounds per square inch, gauge",NaN,"$800,000",NaN,Natural gas distribution,About 8:54 a.m. eastern daylight time1,"Leak, explosion, and fire",NaN,NaN
6,"On December 13, 2005, at 9:26 a.m., an...",DCA-06-MP-001,1 1/4-inch steel service line,NaN,"December 13, 2005",NaN,NaN,3,NaN,5,...,"11 1/2 pounds per square inch, gauge",NaN,NaN,"$863,300",NaN,NaN,9:26 a.m. eastern standard time1,"Pipeline break, explosion, and fire",NaN,Natural gas distribution


In [996]:
df.insert(sum(df.columns.value_counts()), 'Source', value='https://www.ntsb.gov/investigations/AccidentReports/Pages/pipeline.aspx')
df


,Text,accident_no,component_affected,damage_clean_up_cost,date,date_and_time,failure_type,Death,Injury,injuries,...,pressure,product_released,Loss Estimated ($),quantity_released,system_type,time,type_of_accident,type_of_failure,type_of_system,Source
0,,DCA-99-MP-004,¾-inch steel gas service line,NaN,NaN,"January 22, 1999, 10:02 a.m. CST",NaN,3,NaN,6,...,35 psig,NaN,$1.4 million,NaN,NaN,NaN,"Rupture, explosion, and fire",Excavation damage,Natural gas service line,https://www.ntsb.gov/investigations/AccidentRe...
1,"On the afternoon of February 9, 1999, C...",DCA99-MP005,"10-inch-diameter, Grade 5L X-42, 0.250-inch wall",NaN,"February 9, 1999",NaN,NaN,NaN,None,NaN,...,NaN,"Diesel fuel, high sulfur, 86 Grade",$7 million,"About 53,550 gallons (1,275 barrels)",NaN,11:58 p.m. EST,Pipe rupture,NaN,Hazardous liquid petroleum products pipeline,https://www.ntsb.gov/investigations/AccidentRe...
2,"About 12:12 p.m. CST1 on January 27, 2000, a ...",NaN,"606 pounds per square inch, gauge, (psig)",None,NaN,"January 27, 2000, 12:12 p.m. central standard",NaN,NaN,Marathon Ashland Pipe Line LLC,NaN,...,NaN,NaN,NaN,Crude oil,NaN,NaN,Pipe failure and leak,NaN,Hazardous liquid,https://www.ntsb.gov/investigations/AccidentRe...
3,"On March 9, 2000, about 10:20 p.m., central s...",NaN,at site of failure,$18 million,NaN,"March 9, 2000, at 10:20 p.m., central standard...",NaN,NaN,None,NaN,...,NaN,NaN,NaN,"Approximately 564,000 gallons",NaN,NaN,Pipe failure and leak,NaN,Hazardous liquid,https://www.ntsb.gov/investigations/AccidentRe...
4,"On July 2, 2003, a contractor hired by...",DCA-03-MP-004,Excavation damage,NaN,NaN,"July 2, 2003, at 1:44 p.m., eastern daylight t...",NaN,business as Conectiv Power Delivery,NaN,None,...,Natural gas,NaN,14,NaN,NaN,NaN,"Rupture, explosion, and fire",6 to 8 inches water column,Natural gas distribution,https://www.ntsb.gov/investigations/AccidentRe...
5,"On August 21, 2004, about 8:54 a.m., a...",DCA-04-MP-006,2-inch plastic main line,NaN,"August 21, 2004",NaN,Butt-fusion joint leak,2,NaN,None,...,"50 pounds per square inch, gauge",NaN,"$800,000",NaN,Natural gas distribution,About 8:54 a.m. eastern daylight time1,"Leak, explosion, and fire",NaN,NaN,https://www.ntsb.gov/investigations/AccidentRe...
6,"On December 13, 2005, at 9:26 a.m., an...",DCA-06-MP-001,1 1/4-inch steel service line,NaN,"December 13, 2005",NaN,NaN,3,NaN,5,...,NaN,NaN,"$863,300",NaN,NaN,9:26 a.m. eastern standard time1,"Pipeline break, explosion, and fire",NaN,Natural gas distribution,https://www.ntsb.gov/investigations/AccidentRe...


In [999]:
writer = pd.ExcelWriter('pipeline_accident_reports.xlsx')
df.to_excel(writer,'pipeline_accident_reports')
writer.save()